In [ ]:
import os
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
# Read data

folder_fields = "../data/Calibrations/Calibration_top_grey/muDIC"

frames_true = np.load(os.path.join(folder_fields, "frames.npy"))
coords = np.load(os.path.join(folder_fields, "coords.npy"))
strain = np.load(os.path.join(folder_fields, "true_strain.npy"))

with open(os.path.join(folder_fields, 'params.txt')) as file:
    params = file.read()

In [ ]:
# Data shape

print(f"Frames computed: {frames_true.tolist()}")
print(f"Coords of shape {coords.shape} -> (useless, x or y, x_elem, y_elem, frame)")
print(f"Strain of shape {strain.shape} -> (useless, ?, ?, x_elem, y_elem, frame)")

In [ ]:
# Plot one frame

frame = -1
frame_true = frames_true[frame]

x = coords[0,0,:,:,frame]
y = coords[0,1,:,:,frame]

strain_toPlot = strain[0,0,0,:,:,frame]

fig, ax = plt.subplots()
ax.set_title(f"Strain [frame {frame_true}]")

im = ax.contourf(x, y, strain_toPlot)
fig.colorbar(im, ax=ax, label="Stain")

ax.set_xlabel('Position x [pix]')
ax.set_ylabel('Position y [pix]')

plt.show()

In [ ]:
# Plot animation

nbFrames = coords.shape[-1]
frame_start = -1

fig, ax = plt.subplots()

# Set labels
ax.set_title(f"Strain [frame {frames_true[frame_start]}]")
ax.set_xlabel('Position x [pix]')
ax.set_ylabel('Position y [pix]')

# Set bounds
xbounds = coords[0,0,:,0,:].min(), coords[0,0,:,0,:].max()
ybounds = coords[0,1,:,0,:].min(), coords[0,1,0,:,:].max()
ax.set_xbound(*xbounds)
ax.set_ybound(*ybounds)

# Set static cmap
vmin, vmax = strain[0,0,0,:,:,1:].min(), strain[0,0,0,:,:,1:].max()
norm = plt.Normalize(vmin, vmax)
cmap = plt.get_cmap('viridis')
n_levels = 100

# Plot last frame
x = coords[0,0,:,:,frame_start]
y = coords[0,1,:,:,frame_start]
strain_toPlot = strain[0,0,0,:,:,frame_start]

cont = ax.contourf(x, y, strain_toPlot, levels=n_levels, cmap=cmap, norm=norm)
fig.colorbar(cont, ax=ax, label="Strain")


def update(frame):
    global cont

    x = coords[0,0,:,:,frame]
    y = coords[0,1,:,:,frame]
    strain_toPlot = strain[0,0,0,:,:,frame]
    
    for c in cont.collections:
        c.remove()  # removes only the contours, leaves the rest intact

    cont = plt.contourf(x, y, strain_toPlot, levels=n_levels, cmap=cmap, norm=norm)

    # Set limits and title
    ax.set_title(f"Strain [frame {frames_true[frame]}]")
    ax.set_xbound(*xbounds)
    ax.set_ybound(*ybounds)
    
    return cont

anim = animation.FuncAnimation(fig, update, frames=nbFrames, repeat=False, interval=100)
anim_html = HTML(anim.to_html5_video())

print(f"{nbFrames} frames")


Question: Why is the x extend changing ??

In [ ]:
# Display animation

print(type(anim_html))
anim_html

In [ ]:
# Save animation

savepath = os.path.join(folder_fields, "animation_strain.html")
with open(savepath, 'w') as f:
    f.write(anim_html.data)